# 웹 스크래핑 특징

## 웹 페이지의 데이터를 추출 목적

### 여러 웹사이트에서 은밀하게 데이터를 수집

### robots.txt분석하기  - User-agent가 특정 URL에 대해 접근 가능한지 체크

In [1]:
import urllib.robotparser
rp = urllib.robotparser.RobotFileParser()
rp.set_url("https://www.reuters.com/robots.txt")
rp.read()

rp.can_fetch(useragent="*",url="https://reuters.com/sitemap.xml")

True

### 또한 robots.txt에 있는 sitemap에 대한 정도 확인

In [2]:
sitemaps = rp.site_maps()
sitemaps

['https://www.reuters.com/arc/outboundfeeds/sitemap-index/?outputType=xml',
 'https://www.reuters.com/arc/outboundfeeds/news-sitemap-index/?outputType=xml',
 'https://www.reuters.com/sitemap_video_index.xml',
 'https://www.reuters.com/brandfeature/sitemap']

## 사이트 맵을 활용한 웹 크롤러 작성하기

### XML파서 .xmltodict가 XML파서를 위한 모듈이다. 
#### xmltidict의 parse()메서드를 통해 받아온 XML값 (request.get(url).text)을 argument로 전달

In [3]:
import xmltodict
import requests

url = sitemaps[0]
sitemaps = xmltodict.parse(requests.get(url).text)

In [4]:
sitemaps

{'sitemapindex': {'@xmlns': 'http://www.sitemaps.org/schemas/sitemap/0.9',
  'sitemap': [{'loc': 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml',
    'lastmod': '2023-02-28T10:50:03.606Z'},
   {'loc': 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=100',
    'lastmod': '2023-02-28T10:50:03.606Z'},
   {'loc': 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=200',
    'lastmod': '2023-02-28T10:50:03.606Z'},
   {'loc': 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=300',
    'lastmod': '2023-02-28T10:50:03.606Z'},
   {'loc': 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=400',
    'lastmod': '2023-02-28T10:50:03.606Z'},
   {'loc': 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=500',
    'lastmod': '2023-02-28T10:50:03.606Z'},
   {'loc': 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=600',
    'lastmod': '2023-02-28T10:50:03.

### sitemaps에 담긴 데이터를 가지고 'loc'에 해당하는 리스트 생성

In [5]:
[url_list['loc'] for url_list in sitemaps['sitemapindex']['sitemap']]

['https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml',
 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=100',
 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=200',
 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=300',
 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=400',
 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=500',
 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=600',
 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=700',
 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=800',
 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=900',
 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=1000',
 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=1100',
 'https://www.reuters.com/arc/outboundfeeds/sitemap/?output

In [6]:
sitemap_urls = [url_list['loc'] for url_list in sitemaps['sitemapindex']['sitemap']]

### sitemap url중 첫번째 URL만 활용하여 XML파일을 HTTP통신을 통해 받아온다.
### xml파일을 dict로 파싱을 해준다.
#### 신문기사 정보까지 도달

In [7]:
url = sitemap_urls[0]
news_dict = xmltodict.parse(requests.get(url).text)
news_dict

{'urlset': {'@xmlns': 'http://www.sitemaps.org/schemas/sitemap/0.9',
  '@xmlns:image': 'http://www.google.com/schemas/sitemap-image/1.1',
  'url': [{'loc': 'https://www.reuters.com/business/finance/bank-montreal-reports-fall-first-quarter-profit-2023-02-28/',
    'lastmod': '2023-02-28T10:49:50.467Z'},
   {'loc': 'https://www.reuters.com/technology/eu-antitrust-regulators-narrow-case-against-apple-2023-02-28/',
    'lastmod': '2023-02-28T10:48:58.187Z',
    'image:image': {'image:loc': 'https://www.reuters.com/resizer/87loJZj3JUhEXP-JBR32EaJ8Low=/cloudfront-us-east-2.images.arcpublishing.com/reuters/5PGHLP3XFZLD7KKBFX5F7N22ZQ.jpg',
     'image:caption': 'Silhouette of a mobile user seen next to a screen projection of the Apple logo in this picture illustration taken March 28, 2018.  REUTERS/Dado Ruvic/Illustration'}},
   {'loc': 'https://www.reuters.com/world/europe/nikos-christodoulides-new-cyprus-president-sworn-2023-02-28/',
    'lastmod': '2023-02-28T10:47:15.659Z',
    'image:imag

### dict 자료형의 news_dict를 활용해 뉴스의 URL값의 리스트

In [8]:
news_urls = [news_urls['loc'] for news_urls in news_dict['urlset']['url']]

In [9]:
#url을 활용하여 데이터 다운로드하기
#%%time

session = requests.Session()

for url in news_urls[5:8]:
    file = url.split("/")[-2] + '.html'
    
    response = session.get(url)
    if response.ok:
        with open("datas/" +file, "w+b") as f:
            f.write(response.text.encode('utf-8'))
    else:
        print(f"error widh URL : {url}")

## 파일 사용하기
### 파일을 생성 / open() 함수 활용 / 파일에 문자열을 쓸 때는 write()메서드 활용 / argumaent로 작성할 내용 적기 / 닫기

In [10]:
#파일 객체를 준다  / 쓰기모드 w로 열기
file = open('filename.txt','w')

#파일 객체의 write메서드를 통해 문자열을 파일에 쓰기 가능
file.write('파일에 작성할 문자열')

#파일에 대한 처리가 끝나면 반드시 닫기
file.close()

In [11]:
#close에 대한 처리 없이 간편히 쓰려면 with활용
#아래 코드는 이전에 작성했던 것과 동일
with open('filename.txt','w') as file :
    file.write('파일에 작성할 문자열')

In [12]:
# w는 write (wt) / r은 read (rt) / t는 default 
with open('filename.txt','r') as file :
    print(file.read())

파일에 작성할 문자열


## 파일 이름 가져오기
### 지정된 장소의 파일 목록을 가져온다

In [13]:
#os패키지의 listdir()메서드를 활용하면 arguments로 전달된 path안의 파일 목록을 가져온다.
import os
path = "./datas/"
files = [path + file for file in os.listdir(path)]
files

['./datas/.ipynb_checkpoints',
 './datas/abrdn-swings-pretax-loss-markets-turmoil-weighs-2023-02-28.html',
 './datas/api_test.ipynb',
 './datas/atp-roundup-andrey-rublev-avoids-early-upset-dubai-2023-02-28.html',
 './datas/audi-will-not-cut-ev-prices-follow-teslas-lead-audi-europe-chief-2023-02-28.html',
 './datas/chevron-update-production-targets-during-investor-day-2023-02-28.html',
 './datas/china-income-spending-per-capita-grow-over-2022-national-bureau-statistics-2023-02-28.html',
 './datas/filename.txt',
 './datas/filetest.txt',
 './datas/ftse-100-falls-open-ocado-sinks-bottom-2023-02-28.html',
 './datas/indian-security-forces-battle-kashmir-militants-after-killing-hindu-2023-02-28.html',
 './datas/migrant-shipwreck-death-toll-up-64-reuters-witnesses-2023-02-28.html',
 './datas/new-delhi-city-warns-uber-ola-use-bike-taxis-official-2023-02-28.html',
 './datas/payments-firm-klarnas-operating-loss-shrinks-q4-2023-02-28.html',
 './datas/russias-magnit-opens-first-hard-discount-stores

### HTML 파서를 사용한 데이터 추출
### read()메서드를 통해 해당 파일을 읽어오고 그 내용을 html이라는 변수에 담는다.

In [21]:
#HTML 파서를 사용한 데이터 추출
with open(files[1],"r",encoding = 'utf-8') as f:   # 3번째 있는 HTML파일을 가져와
    html = f.read()                                # r모드 (read)모드로 파일을 읽어와 내용을 html변수에 담음

## BeautifulSoup
### json - json
### xml - xnltidict
### html - beautifulsoup

### Beautifulsoup는 HTML 파싱을 하는데 도움을 주는 모듈 
### import Beautifulsoup를 통해 패키지 포함 
### 파서의 종류 “html.parser”, “lxml”, “lxml-xml”, “xml”, “html5lib”

In [23]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html,'html.parser')
title = soup.select_one("h1")
print(title.text)

Abrdn falls to pretax loss as market turmoil weighs


### 해당 태그명을 속성처럼 활용 가능
#### soup.h1이라고 조회하면 \<h1\> 태크에 해당하는 객체를 가져온다
#### soup.h1.text -> 태그 안에 있는 텍스트 값 가져온다

In [25]:
soup.h1.text

'Abrdn falls to pretax loss as market turmoil weighs'

In [26]:
soup.select_one("#main-content > article > div.article__main__33WV2 > div.article__content__6hMn9 > header > div > div > h1").text

'Abrdn falls to pretax loss as market turmoil weighs'

## URL추출

In [27]:
#표준(standard) URL(=Canonical URL)
soup.find("link", {"rel":"canonical"})["href"]

'https://www.reuters.com/markets/europe/abrdn-swings-pretax-loss-markets-turmoil-weighs-2023-02-28/'